# 0. Tabla de contenidos

# 1. Introducción

Cada día, millones de datos se suben a la red sin darnos cuenta. ¿Una opinión? ¿Una sugerencia? Estos datos tienen repercusiones hacia el area que están focalizados. La Bolsa no se ha quedado atrás. Compañías tecnológicas de todo el mundo han visto como las opiniones en RRSS como Twitter puede marcar las futuras acciones de los clientes en las acciones de una compañía, no obstante ejecutivos de iSentium, por ejemplo, comentan que Twitter puede aportar información útil a los inversores, pero que no deberían confiar únicamente en ella a la hora de invertir [1]

Nuestro caso de estudio está enfocado al mercado de valores IBEX35. 

El IBEX 35 es el índice oficial de la bolsa española compuesto por las 35 empresas más negociadas del mercado. Este índice nos muestra en tiempo real si los precios en bolsa están subiendo o bajando, por lo que permite medir el comportamiento de este conjunto de acciones.

El IBEX35 sirve como punto de referencia para los inversores del mercado español. La rentabilidad de este índice es el objetivo a batir por los gestores.

Por lo tanto, la modelización de las dinámicas de este tipo de índices resultan esenciales para la toma de decisiones por parte de todas las entidades bursátiles.

Por lo tanto los objetivos de este estudio son:

✅ Task 1 → Análisis de sentimientos de tweets entre el periodo 2015 hasta la actualidad

✅ Task 2 → ¿Tweeter afecta a la toma de decisiones en la Bolsa IBEX35?

✅ Task 3 → Modelo predictivo de IBEX35


# 2. Preparación de los datos:

La organización de este estudio nos ha proporcionado tres datasets. Dos tienen las mismas características pero uno no tiene el target debido a que se tratarán como datos nuevos. El dataset que resta es un set de tweets con estructura muy diferentes a los otros dos.

## 2.1 Características de los datos:

- `train.csv` - Consta de 6554 entradas y 8 características.

    - `Date` : Día al que hacen referencia los datos presentados.
    - `Open`: Precio de apertura de ese día.
    - `High`: Precio máximo alcanzado durante ese día.
    - `Low`: Precio mínimo alcanzado durante ese día.
    - `Close`: Precio de cierre de ese día ajustado por splits.
    - `Adj Close`: Precio de cierre ajustado por splits y distribuciones de dividendos o plusvalías.
    - `Volume`: El número físico de acciones negociadas del índice bursátil.
    - `Target`: Esta es la variable a predecir. Es una variable binaria.
        - `1`: Indica que el precio de cierre tres días adelante será más alto que el precio de cierre actual.
        - `0`: Indica que el precio de cierre tres días adelante será igual o menor al precio actual.

            
- `test.csv` - Consta de 726 entradas y 7 características.

- `tweets_from2015_#Ibex35.csv`: Contiene los tweets públicos que contienen el hashtag #Ibex35 desde el año 2015 que han recibido más de dos likes y de dos retweets.

## 2.2 Librerías:

Ahora vamos a importar todas las librerías necesarias para hacer este estudio:

In [13]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

from scipy import stats
from scipy.stats import norm

import os
import gc

In [87]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import os
import gc

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import confusion_matrix

from scipy.stats import zscore
from scipy.stats import iqr

from warnings import filterwarnings
filterwarnings('ignore')



os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
import nltk
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata
sentiment_i_a = SentimentIntensityAnalyzer()
import string
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 2.3 Leer los datos:

### 2.3.1 Train y Test:

Como hemos mencionado el dataset `train` y `test` son iguales menos el target que está presente en el dataset train solamente. Vamos a leerlos:

In [15]:
train = pd.read_csv('../input/ibex98/train.csv')
df_test = pd.read_csv('../input/ibex98/test_x(1).csv')

Vemos que el dataset `train` está compuesto por 6554 entradas y 8 caractarísticas. Vemos que hay presencia de datos faltantes, por lo que se deberá corregir posteriormente.

In [16]:
train.info()

In [17]:
df_test.info()

### 2.3.3 Tweet:

Este dataset es diferente en su estructura. Como se puede apreciar consta de 9801 entradas y 3 características. Como podemos observar hay una característica que es similar al dataset train y test, `tweetDate`. Aunque el formato es diferente, podemos modificarlo para que tengan el mismo formato y si es una columna principal (es decir, que determina el conjunto del dataset y por ello no tiene índices duplicados) podemos unirla a la columna principal de train, `Date`. De esta manera podremos ver la implicación de los tweets en las decisiones de compra venta de nuestro inversores en el IBEX35.

In [18]:
tweet = pd.read_csv('../input/ibex98/tweets_from2015_Ibex35.csv')

In [19]:
tweet.info()

## 2.4 Preprocesamiento de los datos.

### 2.4.1 Datos duplicados:

#### 2.4.1.1 Train y Test:

¿Hay presencia de datos duplicados en nuestro dataset?

In [20]:
print('Número de datos duplicados: ')
print('Conjunto df_train:\t', train[train.duplicated()==True].shape[0])
print('Conjunto df_test:\t', df_test[df_test.duplicated()==True].shape[0])

Como se esperaba, no hay datos duplicados en dichos datasets

#### 2.4.1.2 Tweet:

¿Hay presencia de datos dupicados en este dataset?

In [21]:
print('Número de datos duplicados: ')
print('Conjunto df_train:\t', tweet[tweet.duplicated()==True].shape[0])

Hay 16 datos duplicados. Por lo que se deberán eliminarlos.

### 2.4.2 Datos faltantes:

Los datos faltantes son un problema para modelar el algoritmo, por lo que se debe de tener en cuenta cómo tratarlos. 

#### 2.4.2.1 Train y Test:

Primero nos fijaremos en el dataset `train`. Vemos que sí hay presencia de datos faltantes por lo que, como constituyen el 2% de los datos, vamos a proceder a eliminarlos. 

In [22]:
train = train.dropna()

A continuación, expongo el dataset sin datos faltantes. Vemos que se ha reducido a 6421 entradas.

In [23]:
train.info()

En el caso del `test` se puede observar que no hay presencia de datos faltantes, por lo que se queda con las mismas dimensiones.

#### 2.4.2.2 Tweet:

En este dataset sí hay la presencia de datos faltantes. Por lo que eliminaremos dichas filas, quedándos con 4 rows menos.

In [24]:
tweet[tweet.tweetDate.notnull() ==False]

In [25]:
tweet[tweet.text.notnull() ==False]

In [26]:
tweet = tweet.drop(index=[6931,1070,9667,9634])

In [27]:
tweet = tweet.reset_index()

In [28]:
tweet = tweet.drop(['index','handle'],axis=1)

## 2.5 Tratamiento Outliers:

Vamos a ver si hay outliers y si los hay, vamos a eliminarlo añadiendo un informe final de dónde están y el tanto por ciento que implica eliminarlos. No añadiremos la característica `Volume`porque está muy fuera del rango de las otras y no se pueden ver correctament (aunque la gráfica sea interactiva):

In [29]:
df_train = train

In [30]:
df_train.head()

In [31]:
fig = go.Figure()
# Use x instead of y argument for horizontal plot

x0 = df_train['Open']
x1 = df_train['High']
x2 = df_train['Low']
x3 = df_train['Close']
x4 = df_train['Adj Close']
#x5 = df_train['Volume']

x6 = df_train['Target']

fig.update_layout(title_text='Box plot of variables')

fig.add_trace(go.Box(x=x0, name= "Open"))
fig.add_trace(go.Box(x=x1, name = "High"))
fig.add_trace(go.Box(x=x2, name = "Low"))
fig.add_trace(go.Box(x=x3, name = "Close"))
fig.add_trace(go.Box(x=x4, name = "Adj Close"))
#fig.add_trace(go.Box(x=x5, name = "Volume"))
fig.add_trace(go.Box(x=x6, name = "Target"))

fig.show()

Ahora vamos a ver un informe de los outliers, donde están y el tanto por ciento que representa eliminarlos. Como vemos no supera el 10% por tanto, vale la pena eliminarlos.

In [ ]:
df_train = df_train.drop(outlier_list,axis=0).reset_index(drop = True)

Ahora vamos a ver como queda el target después del tratamiento de outliers

In [33]:
df_train[df_train['Target']==0].shape 
df_train[df_train['Target']==1].shape 


labels = ['0','1']
values = [3033,3388]
colors = ['green','lightgreen','gold', 'mediumturquoise', 'darkorange', 'lightgreen']
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(margin = dict(t=25, l=0, r=0, b=0))
fig.update_traces(marker=dict(colors=colors))
fig.update_layout(
    title_text="Distribución del Target despúes del tratamiento de outliers",
    annotations=[dict(text='sin Outlier', x=0.50, y=0.5, font_size=20, showarrow=False)])

fig.show()

## 2.6 Visualización:

Como bien es sabido, los gráficos de IBEX35 y otros mercados de valores aparecen representadas con las velas japonesas. Estas dan una visión muy clara de la tendencia del índice. Puede indicar tiempos intradía como movimientos por minuto, hora... hasta movimientos por meses, trimestres... por lo que se debe especificar qué tiempos se quiere marcar. En nuetro caso, cada vela será un día. La representación general será la siguiente:

In [34]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Ohlc(x=train['Date'],
        open=train['Open'],
        high=train['High'],
        low=train['Low'],
        close=train['Close']))
        
fig.show()

Como podemos observar, la gráfica marca dos colores (rojo para tendencia bajista y verde para tendencia alcista). A priori no vemos un patrón predeterminado, pero sí que la tendencia neutral desde 2015 nos da a entender que la crisis económica tuvo repercusión en este mercado de valores.

Para invertir en un valor, hay diferentes indicadores que nos ayudan a ver tendencias, cambios de tendencia... etc. El más sencillo es media móvil simple (`SMA`). Éste nos indica el precio promedio de un activo durante un número particular de períodos. En nuestro caso hemos creado 3 periodos de tiempo: 5 días, 20 días y 50 días. A continuación veremos cómo se comportan estos indicadores y si realmente sirven para ver cambios de tendencia.

In [35]:
train['SMA5'] = train.Close.rolling(5).mean()
train['SMA20'] = train.Close.rolling(20).mean()
train['SMA50'] = train.Close.rolling(50).mean()

fig = go.Figure(data=[go.Ohlc(x=train['Date'],
            open=train['Open'],
            high=train['High'],
            low=train['Low'],
            close=train['Close'], name = "OHLC"),
            go.Scatter(x=train.Date, y=train.SMA5, line=dict(color='orange', width=1), name="SMA5"),
            go.Scatter(x=train.Date, y=train.SMA20, line=dict(color='green', width=1), name="SMA20"),
            go.Scatter(x=train.Date, y=train.SMA50, line=dict(color='blue', width=1), name="SMA50")])
fig.show()

Para principiantes, es una forma de empezar a ver las tendencias y los cambios de éstas, pero vamos un poco más allá. Ahora usaremos las `EMA`. Estas se diferencian de las `SMA`en que da más peso a los datos de precios más recientes, lo que hace que reaccione más rápidamente a los cambios de precios. En nuestro caso usaremos dos peridod: 5 días y 20 días. 

In [36]:
train['EMA3'] = train.Close.ewm(span=3, adjust=False).mean()
train['EMA5'] = train.Close.ewm(span=5, adjust=False).mean()
train['EMA20'] = train.Close.ewm(span=20, adjust=False).mean()

df_test['EMA5'] = df_test.Close.ewm(span=5, adjust=False).mean()
df_test['EMA20'] = df_test.Close.ewm(span=20, adjust=False).mean()

fig = go.Figure(data=[go.Ohlc(x=train['Date'],
        open=train['Open'],
        high=train['High'],
        low=train['Low'],
        close=train['Close'], name = "OHLC"),
        go.Scatter(x=train.Date, y=train.EMA5, line=dict(color='orange', width=1), name="EMA3"),
        go.Scatter(x=train.Date, y=train.EMA20, line=dict(color='green', width=1), name="EMA20")])
fig.show()

Como podemos comprobar las `EMA` dan mejor intuición en el cambio de precios. Como el target que los patrocinadores nos ha proporcionado tiene que ver con el comportamiento de 3 días anteriores usaremos usa EMA(3) para ver su comportamiento. No obstante, cada tres días es difícil poder ver un cambio significativo de tendencia. Por eso hemos incorporado EMA(5) y EMA(20), ya que representan la semana y el mes (20 días laborables aproximadamente al mes).

A continuación, incorporaremos la característica `Volumen` para ver el número físico de acciones negociadas del índice bursátil. Como se puede observar, a finales de `2010` los valores de esta característica son significativos, mientras que anteriormente no. 

In [37]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# include candlestick with rangeselector
fig.add_trace(go.Candlestick(x=train['Date'],
                open=train['Open'], high=train['High'],
                low=train['Low'], close=train['Close']),
               secondary_y=True)

# include a go.Bar trace for volumes
fig.add_trace(go.Bar(x=train['Date'], y=train['Volume']),
               secondary_y=False)

fig.layout.yaxis2.showgrid=False
fig.show()

A continuacuón, vamos a deficir más indicadores para ver los soportes y techos de los datos a medida que va transcurriendo el tiempo. Para ello, defiremos las bandas de bollinger. `Las bandas de bollinger` miden la volatilidad del mercado y proporciona una gran cantidad de información muy útil para tomar decisiones de compra y venta de activos financieros. Básicamente, si son estrechas indica que el valor está estancado y puede producirse un cambio de tendencia. También nos marcan las tendencias del valor IBEX35 (en nuestro caso). Como podemos ver hay tres indicadores:
- `B_MA`: indica la banda media conformada por la media de los precios `High` + `Low`+ `Close`.
- `BU`: indica el techo del precio.
- `BL` : indica el soporte del precio.

Todo esto para un periodo de tiempo determinado. Vamos a verlo.

In [38]:
def bollinger_bands(df, n, m):
    # takes dataframe on input
    # n = smoothing length
    # m = number of standard deviations away from MA
    
    #typical price
    TP = (df['High'] + df['Low'] + df['Close']) / 3
    # but we will use Adj close instead for now, depends
    
    data = TP
    #data = df['Adj Close']
    
    # takes one column from dataframe
    B_MA = pd.Series((data.rolling(n, min_periods=n).mean()), name='B_MA')
    sigma = data.rolling(n, min_periods=n).std() 
    
    BU = pd.Series((B_MA + m * sigma), name='BU')
    BL = pd.Series((B_MA - m * sigma), name='BL')
    
    df = df.join(B_MA)
    df = df.join(BU)
    df = df.join(BL)
    
    return df

df = bollinger_bands(train, 20, 2)
df.tail()

In [39]:
fig = go.Figure(data=[go.Ohlc(x=train['Date'],
        open=train['Open'],
        high=train['High'],
        low=train['Low'],
        close=train['Close'], name = "OHLC"),
        go.Scatter(x=train.Date, y=train['Adj Close'], line=dict(color='orange', width=1), name="Adj Close"),
        ])
fig.show()

En este punto, ya tenemos una intuición global de nuestro dataset, sus cambios de tendencias y el análisis técnica para poder verlas. 

## 2.7 Descripción estadística de los datos

Como podemos observar todos los datos presentan la misma escala numérica menos `Volumen`y `Target`. 
Hemos añadido tres estadísticos más para ver si conforman datos sesgados y no simétricos. Para ello nos ayudaremos de un `mapa de color` para ver los datos positivos en verde y negativos en rojo.

In [40]:
# mapa de color
def colour_map(value):
    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = "black"
        
    return "color: %s" %color

In [41]:
stats = df_train.describe()
stats.loc['var'] = df_train.var().tolist()
stats.loc['skew'] = df_train.skew().tolist()
stats.loc['kurt'] = df_train.kurtosis().tolist()
stats.style.applymap(colour_map)

Como se observa la varianza es enorme en todas las características. También hay asimetría y sesgo en los datos, por lo que `no conforma una distibución Gaussiana`. Esto lo vemos más adeltante con un gráfico grobal de todas las variables por separado.

In [42]:
fig = go.Figure(data=[go.Scatter(x = df['Date'], y=df['Adj Close'], name = "Adj Close"),
        go.Scatter(x = df['Date'],y=df['BU'], line=dict(color='orange', width=1), name="BU"),
        go.Scatter(x = df['Date'],y=df['BL'], line=dict(color='pink', width=2), name="BL"),
        go.Scatter(x = df['Date'],y=df['B_MA'], line=dict(color='red', width=1), name="B_MA")
        ])
fig.show()

Como podemos observar, las bandas dan cuenta de la tendencia y cambio de tendencia del mercado. Son una fuente de información muy útil para la compra venta de las acciones. Gracias a ellas, por ejemplo, podemos ver que hay un suelo en `6k` que indicaría que si en un futuro volviera los precios por ese suelo habría la posibilidad de comprar ya que subiría. No obstante, para los más profesionales, hay un término usado `break support` que implica que podría romper ese suelo y el valor continuar bajando hasta el siguiente suelo (en nuestro caso es en `3.6k`). 

Para rizar más el rizo, podríamos incluso creear señales de compra y venta. Las más sencillas son las que presentaremos a continuación:

- Si `High` > `BU` : sell signal weekly
- Si `Low` < `BL`: buy signal weekly

Aunque sé que el estudio indica un target definido en datos de 3 días anterior, no tiene nada que ver con estas señales de compra/venta. El target nos dice que si el precio de cierre tres días adelante es más alto que el precio de cierre actual indicar con `1` ya que sería una señal de una posible tendencia alcista. De lo contraria indicaría un posible cambio de tendencia a tendencia bajista por lo que venderíamos las acciones en este punto ya que su valos comenzaría a caer. No obstante, como he dicho antes 3 días no son suficientes para poder hacer una señal fide-digna. Ahora vamos a ver la gráfica con la incorporación de las señales. Como vemos están perfectamente marcadas. Un proyecto a futuro sería crear señales futuras para poder comprar/vender.

In [43]:
def add_signal(df):
    # adds two columns to dataframe with buy and sell signals
    buy_list = []
    sell_list = []
    
    for i in range(len(df['Close'])):
        #if df['Close'][i] > df['BU'][i]:           # sell signal     daily
        if df['High'][i] > df['BU'][i]:             # sell signal     weekly
            buy_list.append(np.nan)
            sell_list.append(df['Close'][i])
        #elif df['Close'][i] < df['BL'][i]:         # buy signal      daily
        elif df['Low'][i] < df['BL'][i]:            # buy signal      weekly
            buy_list.append(df['Close'][i])
            sell_list.append(np.nan)  
        else:
            buy_list.append(np.nan)
            sell_list.append(np.nan)
         
    buy_list = pd.Series(buy_list, name='Buy')
    sell_list = pd.Series(sell_list, name='Sell')
        
    df = df.join(buy_list)
    df = df.join(sell_list)        
     
    return df

def plot_signals(df, ticker):
    # plot price
    plt.figure(figsize=(15,5))
    plt.plot(df['Date'], df['Adj Close'])
    plt.title('Price chart (Adj Close) ' + str(ticker))
    plt.show()

    # plot  values and significant levels
    plt.figure(figsize=(15,5))
    plt.title('Bollinger Bands chart ' + str(ticker))
    plt.plot(df['Date'], df['Adj Close'], label='Adj Close')

    plt.plot(df['Date'], df['High'], label='High', alpha=0.3)
    plt.plot(df['Date'], df['Low'], label='Low', alpha=0.3)

    plt.plot(df['Date'], df['BU'], label='B_Upper', alpha=0.3)
    plt.plot(df['Date'], df['BL'], label='B_Lower', alpha=0.3)
    plt.plot(df['Date'], df['B_MA'], label='B_SMA', alpha=0.3)
    plt.fill_between(df['Date'], df['BU'], df['BL'], color='grey', alpha=0.1)

    plt.scatter(df['Date'], df['Buy'], label='Buy', marker='^')
    plt.scatter(df['Date'], df['Sell'], label='Sell', marker='v')

    plt.legend()

    plt.show()

In [44]:
#### RESAMPLING TO WEEKLY TO CLEAN NOISE
agg_dict = {'Open': 'first',
          'High': 'max',
          'Low': 'min',
          'Close': 'last',
          'Adj Close': 'last',
          'Volume': 'mean'}

# resampled dataframe
# 'W' means weekly aggregation
df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: x.split()[0]), format='%Y-%m-%d') 
df.set_index('Date', inplace=True)
df_agg = df.resample('W').agg(agg_dict)
df_agg = df.reset_index()

In [45]:
df_agg = add_signal(df_agg)
plot_signals(df_agg, "IBEX")

Una vez tengamos toda esta información, vamos a ver cada característica por separado.

## 2.8 EDA

### 2.8.1 Open:

Open indica el precio de apertura de ese día.

In [51]:
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
from scipy import stats
from scipy.stats import norm
import seaborn as sns

In [52]:
# setting some globl config
plt.style.use('fivethirtyeight')
cust_color = ['#fdc029',
'#f7c14c',
'#f0c268',
'#e8c381',
'#dfc498',
'#d4c5af',
'#c6c6c6',
'#a6a6a8',
'#86868a',
'#68686d',
'#4b4c52',
'#303138',
'#171820',
]

In [55]:

plt.rcParams["figure.figsize"] = (14, 14)

In [56]:
def eli_plot(df, feature, title):
    
    # Creating a customized chart. and giving in figsize and everything.
    
    fig = plt.figure(constrained_layout=True)
    
    # creating a grid of 3 cols and 3 rows.
    
    grid = gridspec.GridSpec(ncols=3, nrows=2, figure=fig)

    # Customizing the histogram grid.
    
    ax1 = fig.add_subplot(grid[0, :2])
    
    # Set the title.
    
    ax1.set_title('Histogram')
    
    # plot the histogram.
    
    sns.distplot(df.loc[:, feature],
                 hist=True,
                 kde=True,
                 fit=norm,
                  hist_kws={
                 'rwidth': 0.85,
                 'edgecolor': 'black',
                 'linewidth':.5,
                 'alpha': 0.8},
                 ax=ax1,
                 color=cust_color[0])
    
    ax1.axvline(df.loc[:, feature].mean(), color='Green', linestyle='dashed', linewidth=3)

    min_ylim, max_ylim = plt.ylim()
    ax1.text(df.loc[:, feature].mean()*1.25, max_ylim*0.85, 
             'Mean: {:.2f}'.format(df.loc[:, feature].mean()), 
             color='Green', fontsize='12',
             bbox=dict(boxstyle='round',facecolor='red', alpha=0.5))
    ax1.legend(labels=['Actual','Normal'])
    ax1.xaxis.set_major_locator(MaxNLocator(nbins=12))
    
    
    ax1.annotate(
    # Label and coordinate
    'Unexpected Spike here!', xy=(10000, 0.00025),xytext=(10500, 0.0004) ,
    horizontalalignment="center",
    # Custom arrow
    arrowprops=dict(arrowstyle='simple',lw=1, color='black'), fontsize=8
    )

    # customizing the QQ_plot.
    
    ax2 = fig.add_subplot(grid[1, :2])
    
    # Set the title.
    
    ax2.set_title('Probability Plot')
    
    # Plotting the QQ_Plot.
    stats.probplot(df.loc[:, feature],
                   plot=ax2)
    ax2.get_lines()[0].set_markerfacecolor('#e74c3c')
    ax2.get_lines()[0].set_markersize(12.0)
    ax2.xaxis.set_major_locator(MaxNLocator(nbins=16))

    # Customizing the Box Plot:
    
    ax3 = fig.add_subplot(grid[:, 2])
    # Set title.
    
    ax3.set_title('Box Plot')
    
    # Plotting the box plot.
    
    sns.boxplot(y=feature, data=df, ax=ax3, color=cust_color[0])
    ax3.yaxis.set_major_locator(MaxNLocator(nbins=24))

    plt.suptitle(f'{title}', fontsize=24, fontname = 'monospace', weight='bold')
    
    
def count_dist(df, colname=None, fixlabel=False, f_axis=None, fixlabel_n=None, fixlabel_txt=None, max_idx=30, fontsize=12, palette=cust_color, rotation=45,
              title='X distribution', y_label='', shift=-0.005):
    """A function for counting and displaying categorical variables including percentage texts."""
    fig, ax = plt.subplots()
    sns.barplot(y=df[colname].value_counts().index[:max_idx],
                x=df[colname].value_counts().values[:max_idx], palette=palette, 
                edgecolor='black', linewidth=1.5, saturation = 1.5)
    z=df[colname].value_counts().values[:max_idx]
    for n, i in enumerate(df[colname].value_counts().index[:max_idx]):    
        ax.text(df[colname].value_counts().values[:max_idx][n]+shift, 
                n, #Y location
                s=f'{round(z[n]/df.shape[0]*100,1)}%',                 
                va='center', 
                ha='right', 
                color='white', 
                fontsize=fontsize,
                bbox=dict(boxstyle='round',facecolor='black', alpha=0.5))
    if fixlabel:
        if f_axis == 'x':
            labels = [item.get_text() for item in ax.get_xticklabels()]
            labels[fixlabel_n] = fixlabel_txt
            ax.set_xticklabels(labels)
        else:
            labels = [item.get_text() for item in ax.get_yticklabels()]
            labels[fixlabel_n] = fixlabel_txt
            ax.set_yticklabels(labels)            

    plt.title(title, fontname = 'monospace', weight='bold')
    del z
    
    plt.yticks(fontsize=12,rotation=rotation)
    plt.xlabel("Count", fontname = 'monospace', weight='semibold')
    plt.ylabel(y_label, fontname = 'monospace', weight='semibold')
    plt.show()

In [57]:
eli_plot(df, 'Open', 'Open Distribution\n')

Como se puede observar sigue una distribución no Gaussiana con una media de 8963.54. La densidad es muy pequeña. 

### 2.8.2 High:

High indica el precio de máximo de ese día.

In [58]:
eli_plot(df, 'High', 'High Distribution\n')

Como observamos tampoco sigue una ditribución Gaussiana y la gráfica QQ indica que no los datos varian de la normal.

### 2.8.3 Low:

Low indica el precio mínimo de ese día.

In [59]:
eli_plot(df, 'Low', 'Low Distribution\n')

### 2.8.4 Close:

Close indica el precio de cierre de ese día ajustado por splits

In [60]:
eli_plot(df, 'Close', 'Close Distribution\n')

### 2.8.5 Adj Close:

Adj Close indica el precio de cierre ajustado por splits y distribuciones de dividendos o plusvalías.

In [61]:
eli_plot(df, 'Adj Close', 'Adj Close Distribution\n')

### 2.8.6 Volume:

Volume indica el número físico de acciones negociadas del índice bursátil.

In [62]:
eli_plot(df, 'Volume', 'Volume Distribution\n')

### 2.8.7 Target:

Target: Esta es la variable a predecir. Es una variable binaria

In [63]:
# plotting and styling
fig, ax = plt.subplots(figsize=(12,4))
sns.barplot(x=df['Target'].value_counts().index,
            y=df['Target'].value_counts().values,
            palette=cust_color[::4],
            edgecolor='black', linewidth=1.5, saturation=1.5)
plt.xlabel("Type of Pollutant", fontname = 'monospace', weight='semibold')
plt.ylabel("Count", fontname = 'monospace', weight='semibold')
plt.title('Target Distribution', fontname = 'monospace', weight='bold');

Vemos que este estudio es un caso de `clasificación binaria balanceada`.

## 2.9 Estudio del Target 

Vamos a ver cuáles son las características que tienen mayor correlación con la variable Target:

In [64]:
data = df_train.corr().loc[:,['Target']]
plt.rcParams["figure.figsize"] = (10,3)
# Fetch Index and Values From Data
index = data.index[1:]
values = data.values.flatten()[1:]

# Set figure size, title and labels
fig,ax = plt.subplots(figsize=(30,8))
ax.set_title("Correlación con la variable target\n", size=40)
ax.set_xlabel("Columns", size= 30)
ax.set_ylabel("\nCorrelation", size=30)

# Plot a Barplot
plot = plt.bar(index,values,color=['red' if x<0 else 'green' for x in values])

# Annotate Plots
for p in ax.patches:
    ax.annotate("{:.2f}".format(p.get_height()),(p.get_x(),p.get_height()))

# Show plot
plt.show()

Como podemos observar, ninguna característica tiene una correlación significativa con el target.

## 2.10 Estudiar dataset Tweet:

Vamos a proceder a eliminar de la parte `text` los emogis, cambiar las mayúsculas por minúsculas entre otras funciones para limpiar el texto y poder clasificarlo.

In [65]:
from datetime import datetime, timedelta
import requests
import pandas as pd
from nltk.stem import WordNetLemmatizer

In [66]:
def clean_text(text):
    text = re.sub(r'@[A-Za-z09]+','', text)
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'https?:\/\/?','', text)
    #text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'\n','', text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    return text

tweet['Tweet_new'] = tweet['text'].apply(clean_text)

In [67]:
# Defining dictionary containing all emojis with their meanings.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}


In [68]:
def preprocess(textdata):
    processedText = []
    
    # Create Lemmatizer and Stemmer.
    wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[A-Za-z09]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    for tweet in textdata:
        tweet = tweet.lower()
        
        # Replace all URls with 'URL'
        tweet = re.sub(urlPattern,'',tweet)
        # Replace all emojis.
        for emoji in emojis.keys():
            tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji])        
        # Replace @USERNAME to 'USER'.
        tweet = re.sub(userPattern,'', tweet)        
        # Replace all non alphabets.
        tweet = re.sub(alphaPattern, " ", tweet)
        # Replace 3 or more consecutive letters by 2 letter.
        #tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

        tweetwords = ''
        for word in tweet.split():
            # Checking if the word is a stopword.
            #if word not in stopwordlist:
            if len(word)>1:
                # Lemmatizing the word.
                word = wordLemm.lemmatize(word)
                tweetwords += (word+' ')
            
        processedText.append(tweetwords)
        
    return processedText

In [69]:
import time
t = time.time()
text = tweet['text']
tweet['text_new'] = preprocess(text)
print(f'Text Preprocessing complete.')
print(f'Time Taken: {round(time.time()-t)} seconds')

In [70]:
tweet.head()

Las columnas que nos interesan son `tweetDate` y `text_new`.

In [71]:
tweet = tweet[['tweetDate','text_new']]

A continuación, vamos a eliminar unas filas adicionales de la parte de `tweetDate` debido a que no tienen el formato adecuado.

In [72]:
tweet = tweet.drop([6930,9664,8498,9632])

A continuación, vamos a cambiar el formato de la caracterítica date para ser más clarificante:

In [73]:
weekdayDict = {"0": "M", "1": "Tu", "2": "W", "3": "Th", "4": "F", "5": "Sa", "6": "Su"}

tweet["tweetDate"] = tweet["tweetDate"].astype('datetime64[ns]')
tweet["hour"] = tweet["tweetDate"].apply(lambda x: x.hour)
tweet["day"] = tweet["tweetDate"].apply(lambda x: x.weekday())
tweet["dayofmonth"] = tweet["tweetDate"].apply(lambda x: x.day)
tweet["month"] = tweet["tweetDate"].apply(lambda x: x.month)
tweet["date"] = tweet['tweetDate'].dt.date

In [74]:
tweet.head()

In [75]:
def get_polarity(text):
    analysis = TextBlob(text)
    if text!= '':
        result = analysis.translate(from_lang = 'es', to = 'en').sentiment.polarity

In [ ]:
tweet['polarity'] = tweet['text_new'].apply(get_polarity)

# 3. Algoritmo:

En esta parte del proyecto, vamos a diseñar un algoritmo tipo clasificatorio binario para datos balanceados. Para ello vamos a usar `Bottleneck-Autoencoder`. La idea general será comprimir el espacio de características, permitiendo que el modelo aprenda características más significativas y robustas. La incrustación se añadirá al conjunto de datos original, alimentando la antigua red neuronal totalmente conectada para crear la predicción final. 

Para mayor claridad vamos a esquematizar el algoritmo por pasos:

- `Paso 1`: Seleccionar las caracteríticas a usar:

In [76]:
train_df = df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'EMA5', 'EMA20','Target']]

In [90]:
df_test.columns

In [91]:
df_test_sin_index = df_test[['Open', 'High', 'Low', 'Close', 'Adj Close',
       'Volume', 'EMA5', 'EMA20']]

- `Paso 2`: Dividir nuestro dataset:

In [92]:
# split dataframes for later modeling
X = train_df.drop(['Target'], axis=1).copy()
y = train_df['Target'].copy()

X_test = df_test_sin_index.copy()

In [93]:
print(X.shape, y.shape,X_test.shape)

- `Paso 3`: 

In [94]:
# define helper functions
def set_seed(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    print(f"Seed set to: {seed}")

def plot_eval_results(scores, n_splits):
    cols = 5
    rows = int(np.ceil(n_splits/cols))
    
    fig, ax = plt.subplots(rows, cols, tight_layout=True, figsize=(20,2.5))
    ax = ax.flatten()

    for fold in range(len(scores)):
        df_eval = pd.DataFrame({'train_loss': scores[fold]['loss'], 'valid_loss': scores[fold]['val_loss']})

        sns.lineplot(
            x=df_eval.index,
            y=df_eval['train_loss'],
            label='train_loss',
            ax=ax[fold]
        )

        sns.lineplot(
            x=df_eval.index,
            y=df_eval['valid_loss'],
            label='valid_loss',
            ax=ax[fold]
        )

        ax[fold].set_ylabel('')

    sns.despine()

def plot_cm(cm):
    metrics = {
        'accuracy': cm / cm.sum(),
        'recall' : cm / cm.sum(axis=1),
        'precision': cm / cm.sum(axis=0)
    }
    
    fig, ax = plt.subplots(1,3, tight_layout=True, figsize=(15,5))
    ax = ax.flatten()

    mask = (np.eye(cm.shape[0]) == 0) * 1

    for idx, (name, matrix) in enumerate(metrics.items()):

        ax[idx].set_title(name)

        sns.heatmap(
            data=matrix,
            cmap=sns.dark_palette("#69d", reverse=True, as_cmap=True),
            cbar=False,
            mask=mask,
            lw=0.25,
            annot=True,
            fmt='.2f',
            ax=ax[idx]
        )
    sns.despine()

- `Paso 4`: difinir los callbacks: ReduceLROnPlateau y EarlyStopping

In [95]:
# define callbacks
lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.5, 
    patience=5, 
    verbose=True
)

es = keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    patience=10, 
    verbose=True, 
    mode="min", 
    restore_best_weights=True
)

- `Paso 5`: Crear los autoencoders:

In [96]:
# create autoencoder
class EncodingLayer(layers.Layer):
    def __init__(self, encoding_dim, activation='relu'):
        super().__init__()
        self.enc1 = layers.Dense(encoding_dim*8, activation)
        self.enc2 = layers.Dense(encoding_dim*4, activation)
        self.enc3 = layers.Dense(encoding_dim, activation)
    
    def call(self, inputs):
        x = self.enc1(inputs)
        x = self.enc2(x)
        x = self.enc3(x)
        return x

class DecodingLayer(layers.Layer):
    def __init__(self, encoding_dim, num_outputs, activation='relu'):
        super().__init__()
        self.dec1 = layers.Dense(encoding_dim*4, activation)
        self.dec2 = layers.Dense(encoding_dim*8, activation)
        self.dec3 = layers.Dense(num_outputs, activation='linear')
    
    def call(self, inputs):
        x = self.dec1(inputs)
        x = self.dec2(x)
        x = self.dec3(x)
        return x
    
class AutoEncoder(keras.Model):
    def __init__(self, encoding_dim, num_outputs, activation='relu'):
        super().__init__()
        self.encoder = EncodingLayer(encoding_dim, activation,)
        self.decoder = DecodingLayer(encoding_dim, num_outputs)
    
    def call(self, inputs):
        encoder = self.encoder(inputs)
        decoder = self.decoder(encoder)
        return decoder
    
    def get_encoder(self):
        return self.encoder

- `Paso 6`: Creac capas personalizadas con la clase `DenseBlock` y después conectaremos la red neuronal con la clase `MLP`.

In [97]:
# create custom layer
class DenseBlock(layers.Layer):
    def __init__(self, units, activation='relu', dropout_rate=0, l2=0):
        super().__init__()
        self.dense = layers.Dense(
            units, activation, 
            kernel_initializer="lecun_normal", 
            kernel_regularizer=keras.regularizers.l2(l2)
        )
        self.batchn = layers.BatchNormalization()
        self.dropout = layers.Dropout(dropout_rate)
    
    def call(self, inputs):
        x = self.dense(inputs)
        x = self.batchn(x)
        x = self.dropout(x)
        return x

# create fully-connected NN
class MLP(keras.Model):
    def __init__(self, hidden_layers, autoencoder, activation='relu', dropout_rate=0, l2=0):
        super().__init__()
        self.encoder = autoencoder.get_encoder()
        self.hidden_layers = [DenseBlock(units, activation, l2) for units in hidden_layers]
        self.softmax = layers.Dense(units=target.shape[-1], activation='softmax')
        self.concat = layers.Concatenate()
        
    def call(self, inputs):
        encoding = self.encoder(inputs)
        x = self.concat([inputs, encoding])
        for layer in self.hidden_layers:
            x = layer(x)
        x = self.softmax(x)
        return x

- `Paso 7`: Indicaremos las excepciones del algoritmo.

In [98]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    tf_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU:", tpu.master())
except:
    tf_strategy = tf.distribute.get_strategy()
    print(f"Running on {tf_strategy.num_replicas_in_sync} replicas")
    print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

- `Paso 8`: aplicaremos el cuerpo del algoritmo para predicir el target.

In [102]:
from sklearn.preprocessing import RobustScaler, LabelEncoder

In [104]:
le = LabelEncoder()
target = keras.utils.to_categorical(le.fit_transform(y))
seed = 2022
set_seed(seed)

cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=1)

predictions = []
oof_preds = {'y_valid': list(), 'y_hat': list()}

scores_ae = {fold:None for fold in range(cv.n_splits)}
scores_nn = {fold:None for fold in range(cv.n_splits)}

for fold, (idx_train, idx_valid) in enumerate(cv.split(X,y)):
    X_train, y_train = X.iloc[idx_train], target[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], target[idx_valid]

    # scale data
    scl = RobustScaler()
    X_train = scl.fit_transform(X_train)
    X_valid = scl.transform(X_valid)

    # train autoencoder
    with tf_strategy.scope():
        ae = AutoEncoder(
            encoding_dim=4,
            num_outputs=X.shape[-1],
            activation='relu'
        )

        ae.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-3),
            loss=keras.losses.MeanSquaredError()
        )

    print('_'*65)
    print(f"Fold {fold+1} || Autoencoder Training")

    history_ae = ae.fit(
        X_train, X_train,
        validation_data=(X_valid, X_valid),
        epochs=100,
        batch_size=4096,
        shuffle=True,
        verbose=False,
        callbacks=[lr,es]
    )

    scores_ae[fold] = history_ae.history

    print('_'*65)
    print(f"Fold {fold+1} || AE Min Val Loss: {np.min(scores_ae[fold]['val_loss'])}")

    # train fully-connected nn
    with tf_strategy.scope():
        model = MLP(
            hidden_layers=[384, 256, 128, 64],
            autoencoder=ae,
            activation='selu',
        )

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-3),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=['acc']
        )

    print('_'*65)
    print(f"Fold {fold+1} || NN Training")

    history_nn = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=500,
        batch_size=4096,
        shuffle=True,
        verbose=False,
        callbacks=[lr,es]
    )

    scores_nn[fold] = history_nn.history

    oof_preds['y_valid'].extend(y.iloc[idx_valid])
    oof_preds['y_hat'].extend(model.predict(X_valid, batch_size=4096))

    prediction = model.predict(scl.transform(X_test), batch_size=4096)
    predictions.append(prediction)

    del ae, model
    gc.collect()
    K.clear_session()

    print('_'*65)
    print(f"Fold {fold+1} || NN Min Val Loss: {np.min(scores_nn[fold]['val_loss'])}")

overall_score_ae = [np.min(scores_ae[fold]['val_loss']) for fold in range(cv.n_splits)]
overall_score_nn = [np.min(scores_nn[fold]['val_loss']) for fold in range(cv.n_splits)]

print('_'*65)
print(f"Overall AE Mean Validation Loss: {np.mean(overall_score_ae)} || Overall NN Mean Validation Loss: {np.mean(overall_score_nn)}")

# 4. Evaluación del modelo

Vamos a visualizar los resultados de las funciones de pérdida

In [105]:
plot_eval_results(scores_nn, cv.n_splits)

Ahora vamos a ver, las métricas del modelo para averiguar si el algoritmo a predicho correctamente.

In [106]:
# prepare oof_predictions
oof_y_true = np.array(oof_preds['y_valid'])
oof_y_hat = le.inverse_transform(np.argmax(oof_preds['y_hat'], axis=1))

# create confusion matrix, calculate accuracy, recall & precision
cm = pd.DataFrame(data=confusion_matrix(oof_y_true, oof_y_hat, labels=le.classes_), index=le.classes_, columns=le.classes_)
plot_cm(cm)

Como podemos ver, el `recall`para cuando Target = 1 es del 61% pero si miramos el caso Target = 0 es del 43%. En el Accuracy y la Precisión tienen valores parecidos. La conclusión es que no ha hecho una buena predicción del modelo. Vamos a ver el informe de la matriz de confusión:

In [107]:
cm = confusion_matrix(oof_y_true, oof_y_hat)
ix = np.arange(cm.shape[0])
col_names = [f'Target={cls}' for cls in le.classes_]
cm = pd.DataFrame(cm, columns=col_names, index=col_names)
sns.heatmap(cm, cmap='Blues', annot=True, fmt='d').set(title=f'Matriz de confusión\n');

Como se puede observar, no se ha predicho de forma adecuada. Deberíamos intentar, para próximos estudios, aplicar diferentes ingeniería de características.

In [108]:
from sklearn.metrics import classification_report

In [109]:
print(classification_report(oof_y_true, oof_y_hat))

Ahora vamos a ver los resultados:

In [110]:
#create final prediction, inverse labels to original classes
final_predictions = le.inverse_transform(np.argmax(sum(predictions), axis=1))

In [115]:
df_test.columns

In [116]:
submission = pd.DataFrame()
submission['test_index'] = df_test['test_index']
submission['Prediction'] = final_predictions
submission.head()

In [117]:
submission.to_csv('predictions.csv', index=False)
submission.to_json('predictions.json')

# 5. Conclusiones

Hemos realizado un análisis exaustivo de variables y hemos incorporado indicadores de análisis técnico de bolsa para poder averiguar tendencias, cambios de tendencia y hasta señales de compra y venta. 
Después de desarrollar un algoritmo basado en Autoencoders con keras, hemos podido ver que la clasificación es muy dificultosa. Esto es una muestra más de que predecir los datos a teniendo información de 3 días anteriores es muy dificil, por lo que la mayoría de traders profesionales usan indicadores del cambio de tendencia en periodos de 1 mes, trimestre, semestre... para tener mejor perspetiva y fiabilidad al momento de comprar y vendes acciones en el IBEX35.

# 6. Referencias: 

[1] https://www.expansion.com/mercados/2015/07/13/55a2ab7546163f7c088b4579.html

